## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Green River,US,41.5286,-109.4662,49.19,45,69,8.32,broken clouds
1,1,Mabaruma,GY,8.2000,-59.7833,77.00,94,75,0.00,broken clouds
2,2,Auki,SB,-8.7676,160.7034,77.81,84,40,4.23,scattered clouds
3,3,Singaraja,ID,-8.1120,115.0882,80.73,80,10,0.78,clear sky
4,4,Kodiak,US,57.7900,-152.4072,27.91,46,0,8.05,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Mabaruma,GY,8.2000,-59.7833,77.00,94,75,0.00,broken clouds
2,2,Auki,SB,-8.7676,160.7034,77.81,84,40,4.23,scattered clouds
3,3,Singaraja,ID,-8.1120,115.0882,80.73,80,10,0.78,clear sky
6,6,Honiara,SB,-9.4333,159.9500,77.76,86,45,5.66,scattered clouds
7,7,Isangel,VU,-19.5500,169.2667,80.74,84,100,9.10,overcast clouds
13,13,Mana,GF,5.6592,-53.7767,79.14,86,100,10.94,overcast clouds
25,25,Puerto Ayora,EC,-0.7393,-90.3518,77.85,83,100,3.44,overcast clouds
28,28,Marrakesh,MA,31.6315,-8.0083,82.47,28,0,4.61,clear sky
31,31,Cuamba,MZ,-14.8031,36.5372,80.46,58,45,3.09,scattered clouds
37,37,Chegutu,ZW,-18.1302,30.1407,75.18,54,100,7.92,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        207
City           207
Country        205
Lat            207
Lng            207
Max Temp       207
Humidity       207
Cloudiness     207
Wind Speed     207
Description    207
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#there aren't any empty rows 
#clean_df = preferred_cities_df.dropna(subset = [''])
#clean_df.reset_index(drop=True)

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Mabaruma,GY,77.00,8.2000,-59.7833,
2,Auki,SB,77.81,-8.7676,160.7034,
3,Singaraja,ID,80.73,-8.1120,115.0882,
6,Honiara,SB,77.76,-9.4333,159.9500,
7,Isangel,VU,80.74,-19.5500,169.2667,
13,Mana,GF,79.14,5.6592,-53.7767,
25,Puerto Ayora,EC,77.85,-0.7393,-90.3518,
28,Marrakesh,MA,82.47,31.6315,-8.0083,
31,Cuamba,MZ,80.46,-14.8031,36.5372,
37,Chegutu,ZW,75.18,-18.1302,30.1407,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...skipping.")
        
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [9]:
hotel_df.count()

City          207
Country       205
Max Temp      207
Lat           207
Lng           207
Hotel Name    207
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
# there are no empty rows 
#clean_hotel_df = hotel_df.dropna(subset = ['Hotel Name'])
#clean_hotel_df.reset_index(drop=True)

In [11]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))